In [ ]:
import pandas as pd
from pathlib import Path


import ast

overview_df = pd.read_csv(r"C:\RKT\CODE\python\projects\assignment\Data\movies_overview.csv")
genres_df = pd.read_csv(r"C:\RKT\CODE\python\projects\assignment\Data\movies_genres.csv")


In [8]:
genres_df.head()

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime


In [7]:
overview_df

,title,overview,genre_ids
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[18, 80]"
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]"
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]"
4,12 Angry Men,The defense and the prosecution have rested an...,[18]
...,...,...,...
9995,Driven,Talented rookie race-car driver Jimmy Bly has ...,"[28, 18]"
9996,Home Alone: The Holiday Heist,10-year-old Finn is terrified to learn his fam...,"[35, 10751, 10770]"
9997,Spy Kids 3-D: Game Over,Carmen's caught in a virtual reality game desi...,"[10751, 28, 35, 12, 878]"
9998,My Super Ex-Girlfriend,When New York architect Matt Saunders dumps hi...,"[35, 878, 10749]"


In [10]:
genre_map = dict(zip(genres_df["id"], genres_df["name"]))



In [11]:
overview_df["genre_ids"] = overview_df["genre_ids"].apply(ast.literal_eval)


In [12]:
overview_df["genres"] = overview_df["genre_ids"].apply(
    lambda ids: [genre_map.get(i, "Unknown") for i in ids]
)


In [13]:
print(overview_df[["title", "genres"]].head())


                      title                 genres
0  The Shawshank Redemption         [Drama, Crime]
1             The Godfather         [Drama, Crime]
2     The Godfather Part II         [Drama, Crime]
3          Schindler's List  [Drama, History, War]
4              12 Angry Men                [Drama]


In [62]:
output_path = r"C:\RKT\CODE\python\projects\assignment\Data\final_movie_genre_v2.csv"
overview_df.to_csv(output_path, index=False)


In [61]:
overview_df.head(5)

,title,overview,genre_ids,genres
0,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,"[18, 80]","[Drama, Crime]"
1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]","[Drama, Crime]"
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,"[18, 80]","[Drama, Crime]"
3,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]","[Drama, History, War]"
4,12 Angry Men,The defense and the prosecution have rested an...,[18],[Drama]


In [15]:
final_df=overview_df[["title", "genres"]]

In [16]:
final_df.head()

,title,genres
0,The Shawshank Redemption,"[Drama, Crime]"
1,The Godfather,"[Drama, Crime]"
2,The Godfather Part II,"[Drama, Crime]"
3,Schindler's List,"[Drama, History, War]"
4,12 Angry Men,[Drama]


In [17]:
output_path = r"C:\RKT\CODE\python\projects\assignment\Data\final_movie_genre.csv"
final_df.to_csv(output_path, index=False)


so after just geting the movies and there genres, lets , simply look at what kind of approch we will be able to implement , like for example ,binaay relevnace , well not using this becuase then i wuould i have to train diffrerent classifiers for diffrent classes ,which would be a hasssel , and since , if a movie is from a crime genre , it can be also said that it is a form of a drama film ,,, maybe something like that , so we will be using something called clasification chains 

In [18]:
df=final_df.copy()

In [20]:
df.head()

,title,genres
0,The Shawshank Redemption,"[Drama, Crime]"
1,The Godfather,"[Drama, Crime]"
2,The Godfather Part II,"[Drama, Crime]"
3,Schindler's List,"[Drama, History, War]"
4,12 Angry Men,[Drama]


In [22]:
df.describe()

,title,genres
count,10000,10000
unique,9624,2192
top,Return,[Drama]
freq,13,585


In [28]:
print(df['genres'].dtype)

object


In [29]:
df["genres"] = df["genres"].tolist()

In [30]:
print(df['genres'].dtype)

object


In [31]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df["genres"])
print(mlb.classes_)


['Action' 'Adventure' 'Animation' 'Comedy' 'Crime' 'Drama' 'Family'
 'Fantasy' 'History' 'Horror' 'Music' 'Mystery' 'Romance'
 'Science Fiction' 'TV Movie' 'Thriller' 'War' 'Western']


movies ttilws foras a numerical convert kiya 

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    stop_words="english",
    max_features=500
)

X = tfidf.fit_transform(df["title"])


as usual lets tts (train test split and check various models as well as logging the evaluation (which will be new for me ))

In [34]:
from sklearn.model_selection import train_test_split as tts 
X_train, X_test, Y_train, Y_test = tts(  X, Y, test_size=0.2, random_state=42)


In [35]:
from sklearn.multioutput import ClassifierChain

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import hamming_loss, accuracy_score, f1_score, recall_score


base_lr = LogisticRegression(max_iter=1000)

chain_lr = ClassifierChain(base_lr)
chain_lr.fit(X_train, Y_train)

Y_pred_lr = chain_lr.predict(X_test)


In [37]:
subset_acc = accuracy_score(Y_test, Y_pred_lr)
hamming = hamming_loss(Y_test, Y_pred_lr)

f1_micro = f1_score(Y_test, Y_pred_lr, average="micro")
f1_macro = f1_score(Y_test, Y_pred_lr, average="macro")

recall_micro = recall_score(Y_test, Y_pred_lr, average="micro")
recall_macro = recall_score(Y_test, Y_pred_lr, average="macro")


now logging the metrics with ml flow 

In [38]:
import mlflow
import mlflow.sklearn
mlflow.set_experiment("Movie Genre Classification - Classification Chains")


c:\Users\Boondi ka Ladoo\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026/02/10 10:59:42 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/10 10:59:42 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/10 10:59:42 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/10 10:59:42 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/10 10:59:42 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/10 10:59:42 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/10 10:59:43 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/10 10:59:43 INFO alembic.runtime.migration: Will assume non-transactiona

<Experiment: artifact_location='file:///c:/RKT/CODE/python/projects/assignment/mlruns/1', creation_time=1770701383230, experiment_id='1', last_update_time=1770701383230, lifecycle_stage='active', name='Movie Genre Classification - Classification Chains', tags={}>

In [39]:
with mlflow.start_run(run_name="CC_LogisticRegression"):

    # parameters
    mlflow.log_param("model_type", "Classification Chain")
    mlflow.log_param("base_classifier", "Logistic Regression")
    mlflow.log_param("max_iter", 1000)

    # Log metrics
    mlflow.log_metric("subset_accuracy", subset_acc)
    mlflow.log_metric("hamming_loss", hamming)

    mlflow.log_metric("f1_micro", f1_micro)
    mlflow.log_metric("f1_macro", f1_macro)

    mlflow.log_metric("recall_micro", recall_micro)
    mlflow.log_metric("recall_macro", recall_macro)

    #trained model
    mlflow.sklearn.log_model(chain_lr, artifact_path="model")


2026/02/10 11:00:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\Boondi ka Ladoo\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


well well ,logging for the first  logistic regression as a base classifier is done , lets go for the different one 

In [41]:
# Classification Chain + Naive Bayes
base_nb = MultinomialNB()

chain_nb = ClassifierChain(base_nb)
chain_nb.fit(X_train, Y_train)

Y_pred_nb = chain_nb.predict(X_test)


In [42]:
print("Naive Bayes Chain")
print("Hamming Loss:", hamming_loss(Y_test, Y_pred_nb))
print("Subset Accuracy:", accuracy_score(Y_test, Y_pred_nb))


Naive Bayes Chain
Hamming Loss: 0.14469444444444443
Subset Accuracy: 0.042


In [43]:
subset_acc = accuracy_score(Y_test, Y_pred_nb)
hamming = hamming_loss(Y_test, Y_pred_nb)

f1_micro = f1_score(Y_test, Y_pred_nb, average="micro")
f1_macro = f1_score(Y_test, Y_pred_lr, average="macro")

recall_micro = recall_score(Y_test, Y_pred_nb, average="micro")
recall_macro = recall_score(Y_test, Y_pred_nb, average="macro")

In [44]:
mlflow.set_experiment("Movie Genre Classification - Classification Chains+ Naive Bayes ")
with mlflow.start_run(run_name="CC_Naive Bayes"):

    # parameters
    mlflow.log_param("model_type", "Classification Chain")
    mlflow.log_param("base_classifier", "Naive Bayes")

    # Log metrics
    mlflow.log_metric("subset_accuracy", subset_acc)
    mlflow.log_metric("hamming_loss", hamming)

    mlflow.log_metric("f1_micro", f1_micro)
    mlflow.log_metric("f1_macro", f1_macro)

    mlflow.log_metric("recall_micro", recall_micro)
    mlflow.log_metric("recall_macro", recall_macro)

    #trained model
    mlflow.sklearn.log_model(chain_nb, artifact_path="model_nb")


2026/02/10 11:10:52 INFO mlflow.tracking.fluent: Experiment with name 'Movie Genre Classification - Classification Chains+ Naive Bayes ' does not exist. Creating a new experiment.
2026/02/10 11:10:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\Boondi ka Ladoo\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


well as u can see , the LR+CC haminng loss is 0.14214 and subset accuracy is 0.0865
                    NB+CC haminng loss is 0.14469 and subset accuracy is 0.042

so we wil go for LR+cc model , lets try it 

In [45]:
import time
def predict_genres(input_json, model, vectorizer, mlb):
    """
    Predict genres for a given movie title using
    Classification Chain + Logistic Regression
    """

    # Start timer
    start_time = time.time()

    # Extract title
    title = input_json["title"]

    # Vectorize input
    X_input = vectorizer.transform([title])

    # Predict
    y_pred = model.predict(X_input)

    # Convert binary output to genre names
    predicted_genres = mlb.inverse_transform(y_pred)[0]

    # End timer
    end_time = time.time()

    # Prepare output
    output = {
        "predicted_genres": list(predicted_genres),
        "inference_time": f"{round(end_time - start_time, 2)}s"
    }

    return output








In [51]:
input_data = {
    "title": "Dead Poets Society"
}


In [52]:

result = predict_genres(
    input_json=input_data,
    model=chain_lr,
    vectorizer=tfidf,
    mlb=mlb
)


result

{'predicted_genres': ['Drama'], 'inference_time': '0.01s'}

In [55]:
import time
import numpy as np

def predict_genres_with_threshold( input_json,   model,   vectorizer,   mlb,   threshold=0.3):
    """
    Robust multi-label prediction for
    Classification Chain + Logistic Regression
    """

    start_time = time.time()

    title = input_json["title"]

    # Vectorize input
    X_input = vectorizer.transform([title])

    # Get probabilities
    probas = model.predict_proba(X_input)

    # 🔥 IMPORTANT FIX
    # If output is list (older sklearn behavior)
    if isinstance(probas, list):
        probas = np.array([p[:, 1] for p in probas]).T

    # If output is already ndarray (newer behavior)
    else:
        probas = np.array(probas)

    # Apply threshold
    y_pred = (probas >= threshold).astype(int)

    predicted_genres = mlb.inverse_transform(y_pred)[0]

    end_time = time.time()

    return {
        "predicted_genres": list(predicted_genres),
        "inference_time": f"{round(end_time - start_time, 2)}s"
    }


In [59]:
input_data = {
    "title": "Inception"
}

result = predict_genres_with_threshold(
    input_json=input_data,
    model=chain_lr,
    vectorizer=tfidf,
    mlb=mlb,
    threshold=0.3
)

result


{'predicted_genres': ['Comedy', 'Drama', 'Romance', 'Thriller'],
 'inference_time': '0.01s'}